In [57]:
# Set up your Python workspace
# Note: The STK Python API used in this lesson is
# only available with STK 12.1.
# If not installed then use pip to install it.
# pip install agi.stk<..ver..>-py3-none-any.whl
# If using an older version of STK then use win32api or Comtypes

from agi.stk12.stkdesktop import STKDesktop
from agi.stk12.stkobjects import *

# When connected to STK via Python, while creating your variable, 
# using the Tab key after periods enables IntelliSense which displays 
# all of the options available off of the current interface. 
# In the next section you will start STK 

In [59]:
# NOTE FOR STK WEB: you can take advantage of STK/SDF SSO by changing 
# your script to connect to an active instance instead of creating a 
# new instance of STK:
# Connect to an an instance of STK12
# stk = STKDesktop.AttachToApplication()

# Create a new instance of STK12.
# Optional arguments set the application visible state and the user-control 
# (whether the application remains open after exiting python).
stk = STKDesktop.StartApplication(visible=True, userControl=True)
#Check your Task Manager to confirm that STK was called 

# Grab a handle on the STK application root.
root = stk.Root

# Recall that the AGStkObjectRoot object is at the apex of the STK Object Model. 
# The associated IAgStkObjecetRoot interface will provide the methods and properties to load 
# or create new scenarios and aaccess the Object Model Unit preferences. Through app you have 
# a pointer to the IAgUiApplication interface. How will you obtain a pointer to the IAgStkObjectRoot
# interface? According to IAgUiApplication documentation, the stk.GetObjectRoot() property returns 
# a new instance of the root object of the STK Object Model. 

In [9]:
# Check that the root object has been built correctly, check the type()

type(root)

# output will be 
# agi.stk12.stkobjects.AgStkObjectRoot

agi.stk12.stkobjects.AgStkObjectRoot

In [80]:
# Now that you have launched STK via the Python interface, 
# let's see if we can create a new scenario and set the time 
# period via Python. Create a new scenario, analysis period and 
# reset the animation time.

# 1. Create a new scenario.
# The next task is to create a scenario via the NewScenario method 
# of the IAgStkObjectRoot interface. According to the documentation, 
# the NewScenario method expects to be passed a string representing 
# the name of the scenario, but does not return anything.
root.CloseScenario()

root.NewScenario("STK_Scenario")

scenario = root.CurrentScenario

# 2. Set the analytical time period.

scenario.SetTimePeriod('Today','+24hr')

# 3. Reset the animation time to the newly established start time.

root.Rewind();

In [83]:
# With a new scenario created, it's time to populate the scenario 
# with objects. Use the STK Python API

# Set units so we don't get confused here
root.UnitPreferences.Item('LatitudeUnit').SetCurrentUnit('deg')
root.UnitPreferences.Item('LongitudeUnit').SetCurrentUnit('deg')

# 1. Add a facility
facility = scenario.Children.New(AgESTKObjectType.eFacility, "MyFacility")
facility.Position.AssignGeodetic(38.9072, -77.0369, 0)  # Latitude, Longitude, Altitude of DC
facility.UseTerrain = True

In [91]:
# 2. Add sensors
#sensor1 = facility.Children.New(AgESTKObjectType.eSensor,'Sensor1')
sensor1.Graphics.FillVisible = True
sensor1.Graphics.PercentTranslucency = 50

facility2 = scenario.Children.New(AgESTKObjectType.eFacility, "MyFacility2")
facility2.Position.AssignGeodetic(35, -70, 0)  # Latitude, Longitude, Altitude of DC
facility2.UseTerrain = True

sensor2 = facility2.Children.New(AgESTKObjectType.eSensor,'Sensor2')
#sensor2.LocationData.AssignGeodetic(45, 25, 0)
sensor2.Graphics.FillVisible = True
sensor2.Graphics.PercentTranslucency = 50

# sensor3 = facility.Children.New(AgESTKObjectType.eSensor,'Sensor3')
# #sensor3.LocationData.AssignGeodetic(50, 30, 0)
# sensor3.Graphics.FillVisible = True
# sensor3.Graphics.PercentTranslucency = 50

# sensor4 = facility.Children.New(AgESTKObjectType.eSensor,'Sensor4')
# #sensor4.Position.AssignGeodetic(55, 35, 0)
# sensor4.Graphics.FillVisible = True
# sensor4.Graphics.PercentTranslucency = 50

In [95]:
# Take a moment to create a LEO satellite using Python and
# STK Connect commands, via the ExecuteCommand method.

# 3. Add a Aircraft object to the scenario.
#aircraft1 = scenario.Children.New(AgESTKObjectType.eAircraft, 'Aircraft1')  # eAircraft
help(aircraft1)

Help on AgAircraft in module agi.stk12.stkobjects object:

class AgAircraft(IAgAircraft, IAgStkObject, IAgLifetimeInformation, IAgGreatArcVehicle, IAgProvideSpatialInfo)
 |  AgAircraft(sourceObject=None)
 |  
 |  Aircraft object.
 |  
 |  Method resolution order:
 |      AgAircraft
 |      IAgAircraft
 |      IAgStkObject
 |      IAgLifetimeInformation
 |      IAgGreatArcVehicle
 |      IAgProvideSpatialInfo
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Checks equality of the underlying STK references.
 |  
 |  __init__(self, sourceObject=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __setattr__(self, attrname, value)
 |      Implement setattr(self, name, value).
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __hash__ = None
 |  
 |  ----------------------------------------------------------------------
 |  Readonly 

In [7]:
# Examine the below connect command before running. In it we will be using the
# Set State Classical connect command. Rather than manually setting the times
# we will use the define scenario times. Print them to confirm. 

print(scenario.StartTime)
print(scenario.StopTime)

# 4. Propagate the Aircraft object's orbit with arbitrary waypoints
route = aircraft.Route
ptsArray = [[37.5378, 14.2207, 3.0480, 0.0772, 2], [47.2602, 30.5517, 3.0480, 0.0772, 2]]
route.SetPointsSmoothRateAndPropagate(ptsArray)

# Propagate the route
route.Propagate()

11 Nov 2022 00:00:00.000
12 Nov 2022 00:00:00.000


In [8]:
# Now that the scenario is fully built, the final task is to extract data and perform a basic analysis.
# You have a scenario with a Target object and a Satellite object. 
# Determine when the Satellite object can access the Target object.
# Locate and manually enter code into Python to compute an access between 
# two STK Objects using the IAgStkObject interface. 
# HINT: Review the Python Code snippits
# http://help.agi.com/stkdevkit/index.htm#stkObjects/ObjModPythonCodeSamples.htm

# Enter and run the lines of code to compute access between the satellite and the target

access = satellite.GetAccessToObject(target)
access.ComputeAccess();

In [9]:
# Retrieve and view the access data in Python.

accessDP         = access.DataProviders.Item('Access Data')

results          = accessDP.Exec(scenario.StartTime, scenario.StopTime)

accessStartTimes = results.DataSets.GetDataSetByName('Start Time').GetValues()

accessStopTimes  = results.DataSets.GetDataSetByName('Stop Time').GetValues()

print(accessStartTimes,accessStopTimes)

['11 Nov 2022 00:15:49.041431922', '11 Nov 2022 01:49:13.979975383', '11 Nov 2022 03:28:53.048365111', '11 Nov 2022 05:12:53.435598890', '11 Nov 2022 12:21:03.702464795', '11 Nov 2022 13:59:41.887646050', '11 Nov 2022 15:40:45.244868931', '11 Nov 2022 17:22:58.110913212'] ['11 Nov 2022 00:18:18.436506142', '11 Nov 2022 02:02:55.334556147', '11 Nov 2022 03:44:31.252580703', '11 Nov 2022 05:24:58.704025153', '11 Nov 2022 12:26:24.027626339', '11 Nov 2022 14:14:15.270571053', '11 Nov 2022 15:56:07.470152130', '11 Nov 2022 17:33:42.254177610']


In [10]:
# Note: Generating the Start & Stop times in Python can 
# also be done using the following lines of code.
accessIntervals = access.ComputedAccessIntervalTimes

dataProviderElements = ['Start Time', 'Stop Time']

for i in range(0,accessIntervals.Count):
    times = accessIntervals.GetInterval(i)
    print(times)

('11 Nov 2022 00:15:49.041', '11 Nov 2022 00:18:18.437')
('11 Nov 2022 01:49:13.980', '11 Nov 2022 02:02:55.335')
('11 Nov 2022 03:28:53.048', '11 Nov 2022 03:44:31.253')
('11 Nov 2022 05:12:53.436', '11 Nov 2022 05:24:58.704')
('11 Nov 2022 12:21:03.702', '11 Nov 2022 12:26:24.028')
('11 Nov 2022 13:59:41.888', '11 Nov 2022 14:14:15.271')
('11 Nov 2022 15:40:45.245', '11 Nov 2022 15:56:07.470')
('11 Nov 2022 17:22:58.111', '11 Nov 2022 17:33:42.254')


In [11]:
# Retrieve and view the altitude of the satellite during an access interval.

satelliteDP       = satellite.DataProviders.Item('LLA State')

satelliteDP2      = satelliteDP.Group.Item('Fixed')

rptElements       = ['Time', 'Lat', 'Lon', 'Alt']

satelliteDPTimeVar = satelliteDP2.ExecElements(accessStartTimes,accessStopTimes, 60, rptElements)

satelliteAltitude = satelliteDPTimeVar.DataSets.GetDataSetByName('Alt').GetValues()

print(satelliteAltitude)

[836.6900008063624, 837.8760336816506, 838.9811055930059, 839.4881787430944]


In [12]:
# You have just completed the STK integration with Python tutorial using Jupyter Notebooks. 
# Don't forget to save your work. 

# Note: To save your scenario you will want to create a new directory 
# and then tell python to save all the scenario files there.
import os
os.mkdir('Python_STK_Training')

# This will create a new folder in your scenario directory. 
# Open a file explorer window and copy the file path. The next 
# line will use your custom path.

directory = os.getcwd() + "\Python_STK_Training"
root.ExecuteCommand('Save / * \"' + directory + '\"');

# With your scenario safely saved, you can close out of STK. 
# Workflows like these can be expanded and automated for quickly building and analyzing missions.